In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_boston
import xgboost
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import explained_variance_score
from lightgbm import LGBMRegressor, plot_importance
import lightgbm as lgb
from lightgbm import LGBMClassifier, LGBMRegressor

In [47]:
knobs_rdb = {
#     "appendonly": None,
#     "appendfsync": None,
#     "auto-aof-rewrite-percentage": None,
#     "auto-aof-rewrite-min-size": None,
#     "no-appendfsync-on-rewrite": None,
#     "aof-rewrite-incremental-fsync": None,
#     "aof-use-rdb-preamble": None,

    # "save": None,
    "rdbcompression": None,
    "rdbchecksum": None,
    "rdb-save-incremental-fsync": None,

    "activedefrag": None,
    "active-defrag-threshold-lower": None,
    "active-defrag-threshold-upper": None,
    "active-defrag-cycle-min": None,
    "active-defrag-cycle-max": None,

    # "maxmemory": None,
    "maxmemory-policy": None,
    "maxmemory-samples": None,
    "lazyfree-lazy-eviction": None,
    "lazyfree-lazy-expire": None,
    "lazyfree-lazy-server-del": None,

    "hash-max-ziplist-entries": None,
    "hash-max-ziplist-value": None,
    "activerehashing": None,
    "hz": None,
    "dynamic-hz": None
}

knob_list = list(knobs_rdb.keys())

In [48]:
knob_list

['rdbcompression',
 'rdbchecksum',
 'rdb-save-incremental-fsync',
 'activedefrag',
 'active-defrag-threshold-lower',
 'active-defrag-threshold-upper',
 'active-defrag-cycle-min',
 'active-defrag-cycle-max',
 'maxmemory-policy',
 'maxmemory-samples',
 'lazyfree-lazy-eviction',
 'lazyfree-lazy-expire',
 'lazyfree-lazy-server-del',
 'hash-max-ziplist-entries',
 'hash-max-ziplist-value',
 'activerehashing',
 'hz',
 'dynamic-hz']

## local setting

In [2]:
# config data(.csv) -> DataFrame
config_df = pd.read_csv("./result_config10001_11000.csv", sep=',')

In [64]:
config_df

,appendonly,appendfsync,auto-aof-rewrite-percentage,auto-aof-rewrite-min-size,no-appendfsync-on-rewrite,aof-rewrite-incremental-fsync,aof-use-rdb-preamble,rdbcompression,rdbchecksum,rdb-save-incremental-fsync,...,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,no,...,volatile-lfu,7,no,yes,no,636,235,yes,7,no
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,no,...,volatile-random,3,no,yes,yes,392,67,yes,21,yes
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,yes,...,volatile-lfu,7,yes,yes,yes,699,94,yes,37,yes
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,yes,no,...,volatile-lfu,7,yes,no,yes,400,136,no,9,no
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,no,...,volatile-lru,7,yes,no,no,663,218,no,12,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,NaN,NaN,NaN,NaN,NaN,NaN,NaN,no,no,yes,...,allkeys-lfu,3,yes,yes,no,626,149,yes,13,no
996,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,no,...,volatile-random,6,yes,yes,yes,745,129,yes,31,yes
997,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,no,no,...,volatile-lfu,6,yes,yes,yes,302,101,no,12,yes
998,NaN,NaN,NaN,NaN,NaN,NaN,NaN,yes,yes,yes,...,volatile-lfu,4,no,yes,yes,291,67,yes,19,yes


In [66]:
config_df['activedefrag'].unique()

array(['yes', nan], dtype=object)

In [4]:
# rdb knob으로 분류
config_df_rdb = config_df.loc[:,'rdbcompression':'dynamic-hz']

In [5]:
config_df_rdb.head()

,rdbcompression,rdbchecksum,rdb-save-incremental-fsync,activedefrag,active-defrag-threshold-lower,active-defrag-threshold-upper,active-defrag-cycle-min,active-defrag-cycle-max,maxmemory,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz
0,no,no,no,yes,7.0,83.0,12.0,83.0,NaN,volatile-lfu,7,no,yes,no,636,235,yes,7,no
1,no,no,no,NaN,NaN,NaN,NaN,NaN,NaN,volatile-random,3,no,yes,yes,392,67,yes,21,yes
2,no,no,yes,yes,28.0,93.0,28.0,86.0,NaN,volatile-lfu,7,yes,yes,yes,699,94,yes,37,yes
3,no,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,volatile-lfu,7,yes,no,yes,400,136,no,9,no
4,yes,yes,no,NaN,NaN,NaN,NaN,NaN,NaN,volatile-lru,7,yes,no,no,663,218,no,12,no


In [6]:
# external data(.csv) -> DataFrame
external_df = pd.read_csv("./result_rdb_external_4.csv", sep=',')

In [7]:
# throuput 값으로 예상
external_df['Gets_KB/sec']

0      68121.32
1      67585.45
2      67425.93
3      67046.94
4      66755.71
         ...   
995    67567.17
996    68655.48
997    67605.50
998    67812.35
999    66795.58
Name: Gets_KB/sec, Length: 1000, dtype: float64

## 데이터 전처리

In [8]:
categorical_var = ['rdbcompression', 'rdbchecksum','rdb-save-incremental-fsync', 'activedefrag', 'maxmemory-policy','lazyfree-lazy-eviction','lazyfree-lazy-expire','lazyfree-lazy-server-del','activerehashing', 'dynamic-hz']

for cate_var in categorical_var:
    config_df_rdb[cate_var] = config_df_rdb[cate_var].astype('category') 


In [51]:
# maxmemrory 생략
config_df_rdb = config_df_rdb.drop('maxmemory', axis=1)

In [52]:
config_data = config_df_rdb
throughput_data = external_df['Gets_KB/sec']

In [53]:
config_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 18 columns):
 #   Column                         Non-Null Count  Dtype   
---  ------                         --------------  -----   
 0   rdbcompression                 1000 non-null   category
 1   rdbchecksum                    1000 non-null   category
 2   rdb-save-incremental-fsync     1000 non-null   category
 3   activedefrag                   510 non-null    category
 4   active-defrag-threshold-lower  510 non-null    float64 
 5   active-defrag-threshold-upper  510 non-null    float64 
 6   active-defrag-cycle-min        510 non-null    float64 
 7   active-defrag-cycle-max        510 non-null    float64 
 8   maxmemory-policy               1000 non-null   category
 9   maxmemory-samples              1000 non-null   int64   
 10  lazyfree-lazy-eviction         1000 non-null   category
 11  lazyfree-lazy-expire           1000 non-null   category
 12  lazyfree-lazy-server-del       1000

# 모델 생성

In [54]:
# train, test 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(config_data, throughput_data ,test_size=0.2)


In [55]:
fit_params={"early_stopping_rounds":10, 
            "eval_metric" : 'auc', 
            "eval_set" : [(X_test,y_test)],
            'eval_names': ['valid'],
            'verbose': 100,
            'feature_name': 'auto', # that's actually the default
            'categorical_feature': 'auto' # that's actually the default
           }

In [56]:
lgbr = lgb.LGBMRegressor(num_leaves= 15, max_depth=-1, 
                         random_state=314, 
                         silent=True, 
                         metric='None', 
                         n_jobs=4, 
                         n_estimators=1000,
                         colsample_bytree=0.9,
                         subsample=0.9,
                         learning_rate=0.1)


In [57]:
lgbr.fit(X_train, y_train, **fit_params)

Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[1]	valid's auc: 1


/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/Users/joseong-un/opt/anaconda3/lib/python3.8/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


LGBMRegressor(colsample_bytree=0.9, metric='None', n_estimators=1000, n_jobs=4,
              num_leaves=15, random_state=314, subsample=0.9)

In [58]:
y_predict = lgbr.predict(X_test)

---

In [59]:
# none 존재여부 확인
config_data.head()

# for i in knob_list:
#     print(config_data[i].unique)
config_data.isnull().sum()

rdbcompression                     0
rdbchecksum                        0
rdb-save-incremental-fsync         0
activedefrag                     490
active-defrag-threshold-lower    490
active-defrag-threshold-upper    490
active-defrag-cycle-min          490
active-defrag-cycle-max          490
maxmemory-policy                   0
maxmemory-samples                  0
lazyfree-lazy-eviction             0
lazyfree-lazy-expire               0
lazyfree-lazy-server-del           0
hash-max-ziplist-entries           0
hash-max-ziplist-value             0
activerehashing                    0
hz                                 0
dynamic-hz                         0
dtype: int64

# Objective Func-목적함수

In [61]:
X_test.head()

,rdbcompression,rdbchecksum,rdb-save-incremental-fsync,activedefrag,active-defrag-threshold-lower,active-defrag-threshold-upper,active-defrag-cycle-min,active-defrag-cycle-max,maxmemory-policy,maxmemory-samples,lazyfree-lazy-eviction,lazyfree-lazy-expire,lazyfree-lazy-server-del,hash-max-ziplist-entries,hash-max-ziplist-value,activerehashing,hz,dynamic-hz
58,no,yes,yes,NaN,NaN,NaN,NaN,NaN,volatile-lru,5,no,yes,no,526,32,yes,4,yes
168,yes,yes,no,yes,4.0,80.0,19.0,89.0,noeviction,4,no,no,no,493,95,no,37,yes
128,yes,no,no,NaN,NaN,NaN,NaN,NaN,allkeys-random,4,yes,yes,no,419,159,yes,6,yes
797,no,yes,no,yes,17.0,100.0,18.0,79.0,volatile-ttl,5,no,yes,yes,313,22,no,23,yes
419,yes,no,yes,yes,4.0,97.0,27.0,77.0,volatile-lfu,7,no,no,no,485,155,no,17,no


In [62]:
# 테스트
knob_value = list(X_test.loc[58])

In [72]:
knob_value = ['no',
 'yes',
 'yes',
 'no',
 4.0,
 80.0,
 19.0,
 89.0,
 'volatile-lru',
 5,
 'no',
 'yes',
 'no',
 526,
 32,
 'yes',
 4,
 'yes']

In [84]:
noise_level = 0.1

def f(knob_value, noise_level=noise_level):
    knob_dict = {}

    for i, knob in enumerate(knob_list): # 딕셔너리로 이동
        knob_dict[knob] = knob_value[i]
    
    if knob_dict['activedefrag'] == 'no':  # activedefrag no인 경우 나머지 관련 knob들에 0할당!
        activ_var = ['active-defrag-threshold-lower', 'active-defrag-threshold-upper', 'active-defrag-cycle-min', 'active-defrag-cycle-max']
        for activ_knob in activ_var:
            knob_dict[activ_knob] = 0     
    
    knob_df = pd.DataFrame(knob_dict, index=[0])  # 데이터 프레임으로 변환
    
    categorical_var = ['rdbcompression', 'rdbchecksum','rdb-save-incremental-fsync', 'activedefrag', 'maxmemory-policy','lazyfree-lazy-eviction','lazyfree-lazy-expire','lazyfree-lazy-server-del','activerehashing', 'dynamic-hz']
    for cate_var in categorical_var:  # 카테고리 형으로 변환
        knob_df[cate_var] = knob_df[cate_var].astype('category')

    return -lgbr.predict(knob_df)[0]


#     change_numeric = ['active-defrag-threshold-lower', 'active-defrag-threshold-upper', 'active-defrag-cycle-min', 'active-defrag-cycle-max']


In [93]:
# knob 바운드
knob_bound = [
    # (rdbcompression), (rdbchecksum), (rdb-save-incremental-fsync)
    ('yes', 'no'), ('yes', 'no'), ('yes', 'no'),
    
    # (activedefrag), (active-defrag-threshold-lower), (active-defrag-threshold-upper), (active-defrag-cycle-min), (active-defrag-cycle-max)
    ('yes', 'no'), (1, 31), (70, 101), (1, 31), (70, 91),
    
    # maxmemory -> 수정 필요!
    ## (maxmemmory-policy),  
    ("volatile-lru", "allkeys-lru", "volatile-lfu", "allkeys-lfu", "volatile-random","allkeys-random", "volatile-ttl", "noeviction"),
    ## (maxmemory-samples)
    (3, 7),
    
    #(lazyfree-lazy-eviction), (lazyfree-lazy-expire), (lazyfree-lazy-server-del)
    ('yes', 'no'), ('yes', 'no'), ('yes', 'no'),
    
    
    # (hash-max-ziplist=entries), (hash-max-ziplist-value)
    (256, 751), (16, 257),
    
    # activerehashing, hz, dynamic-hz
    ('yes', 'no'), (1, 41), ('yes', 'no')
]

## gp_minimize

In [97]:
%matplotlib inline
print(__doc__)

import numpy as np
np.random.seed(237)
import matplotlib.pyplot as plt
from skopt.plots import plot_gaussian_process
from skopt import gp_minimize

Automatically created module for IPython interactive environment


In [98]:
res = gp_minimize(f,                  # the function to minimize
                  knob_bound,      # the bounds on each dimension of x
                  acq_func="EI",      # the acquisition function
                  n_calls=15,         # the number of evaluations of f
                  n_random_starts=5,  # the number of random initialization points
                  noise=0.1**2,       # the noise level (optional)
                  random_state=1234)   # the random seed

In [103]:
res.x

['yes',
 'no',
 'no',
 'yes',
 21,
 98,
 31,
 70,
 'volatile-ttl',
 7,
 'yes',
 'no',
 'yes',
 563,
 104,
 'yes',
 41,
 'no']

In [104]:
res.fun # 65000

-67572.36168244839